                                             Домашнє завдання N 3

                                        Лінійна регресія та її реалізація

Імпортуємо потрібні бібліотеки:

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

Завантаження та вивід перших рядків датасету Housing.csv:

In [3]:
housing_data = pd.read_csv('Housing.csv')

housing_data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


Знаходимо найкращі параметри w(омега) для датасету прогнозуючу ціну на будинок залежно
 від площі, кількості ванних кімнат та кількості спалень:

In [4]:
# Вибір ознак і цільової змінної
X = housing_data[['area', 'bedrooms', 'bathrooms']].values
y = housing_data['price'].values

# Нормалізація ознак
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_with_bias = np.c_[np.ones(X_scaled.shape[0]), X_scaled]


Відображення перших 5 рядків нормалізованих даних з одиницями:

In [5]:
X_scaled_with_bias[:5]

array([[ 1.        ,  1.04672629,  1.40341936,  1.42181174],
       [ 1.        ,  1.75700953,  1.40341936,  5.40580863],
       [ 1.        ,  2.21823241,  0.04727831,  1.42181174],
       [ 1.        ,  1.08362412,  1.40341936,  1.42181174],
       [ 1.        ,  1.04672629,  1.40341936, -0.57018671]])

Створимо функцію для обчислення функції втрат у векторному вигляді:

In [6]:
def cost(h, y):
  return np.mean((h-y)**2)/2

Створимо клас для лінійної регресії CustomLinearRegression:

In [7]:
class CustomLinearRegression:
  def __init__(self, lr: float = 0.01, thr=0.00001, n_epochs = 100):
    self.lr = lr
    self.thr = thr
    self.n_epochs = n_epochs
    self.w = np.random.rand(X_scaled_with_bias.shape[1])

  # Функція гіпотези лінійної регресії у векторному вигляді
  def predict(self, X):
    return self.w@X.T
  
  # Один крок градієнтного спуску
  def update_weights(self, X, y):
    m = len(y)
    h = self.predict(X)
    self.w = self.w - self.lr / m * X.T @ (h-y)

  # Навчання моделі та виведення вартості функції втрат під час навчання
  def fit(self, X, y):
    last_cost = 10000000
    for i in range(self.n_epochs):
      self.update_weights(X, y)
      new_cost = cost(self.predict(X), y)

      if i % (self.n_epochs//100) == 0:
        print(f"cost function = {last_cost}")

      if abs(last_cost - new_cost) < self.thr:
        break

      last_cost = new_cost

Ініціалізація та тренування CustomLinearRegression моделі:

In [8]:
cust_regression = CustomLinearRegression(lr=0.01, thr=0.00001, n_epochs=1000)
w_cust_regression = cust_regression.fit(X_scaled_with_bias, y)
w_cust_regression = cust_regression.w

cost function = 10000000
cost function = 10827794033917.678
cost function = 8978438089031.682
cost function = 7476706598948.053
cost function = 6256434779873.583
cost function = 5264256586702.534
cost function = 4457078112394.562
cost function = 3800062176138.666
cost function = 3265017703230.626
cost function = 2829110339431.859
cost function = 2473828521562.638
cost function = 2184153100409.381
cost function = 1947889466312.3572
cost function = 1755129640417.0142
cost function = 1597818486947.3064
cost function = 1469403475749.3835
cost function = 1364551590066.7368
cost function = 1278920272313.2275
cost function = 1208971917014.5317
cost function = 1151823500228.6135
cost function = 1105124591860.2249
cost function = 1066958319909.2323
cost function = 1035760913268.678
cost function = 1010256296785.2294
cost function = 989402891902.8859
cost function = 972350322278.8374
cost function = 958404163156.3223
cost function = 946997227314.5275
cost function = 937666166028.8615
cost functi

Знаходимо ці ж параметри за допомогою аналітичного рішення:

In [9]:
def analytical_solution(X, y):
    return np.linalg.pinv(X) @ y

Застосування аналітичного рішення для пошуку найкращих параметрів:

In [10]:
w_analytical = analytical_solution(X_scaled_with_bias, y)

Порівняймо отримані результати:

In [11]:
f"Найкращі параметри за допомогою методу градієнтного спуску - {w_cust_regression}"


'Найкращі параметри за допомогою методу градієнтного спуску - [4766523.46208285  821199.26714076  300296.2857042   695515.99611495]'

In [12]:
f"Найкращі параметри за допомогою аналітичного рішення - {w_analytical}"

'Найкращі параметри за допомогою аналітичного рішення - [4766729.24770642  821214.14349519  299983.57107963  695808.52272537]'